<a href="https://colab.research.google.com/github/deltorobarba/machinelearning/blob/master/vqe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Variational Quantum Eigensolver (VQE) for Quantum Molecule Simulation**

In [ ]:
!pip install cirq --quiet
import cirq
import sympy
from cirq.contrib.svg import SVGCircuit
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt # %matplotlib inline
print(cirq.__version__)

https://arxiv.org/abs/2012.09265

https://pennylane.ai/qml/demos/tutorial_quantum_natural_gradient.html

https://pennylane.ai/qml/glossary/variational_circuit.html#glossary-variational-circuit

https://pennylane.ai/qml/glossary/parameter_shift.html

https://de.wikipedia.org/wiki/Downhill-Simplex-Verfahren

#### <font color="blue">**Variational Methods for approaching the ground state energy of a system**

###### **Variational Principle**

https://en.m.wikipedia.org/wiki/Many-body_problem

In science and especially in mathematical studies, a [variational principle](https://en.m.wikipedia.org/wiki/Variational_principle) is one that enables a problem to be solved using calculus of variations, which concerns finding functions that optimize the values of quantities that depend on those functions.

In [quantum mechanics, the variational method](https://en.m.wikipedia.org/wiki/Variational_method_(quantum_mechanics)) is one way of finding approximations to the lowest energy eigenstate or ground state, and some excited states. This allows calculating approximate wavefunctions such as molecular orbitals. The basis for this method is the variational principle.

  * The method consists of choosing a "trial wavefunction" depending on one or more parameters, and finding the values of these parameters for which the expectation value of the energy is the lowest possible. 
  
  * The wavefunction obtained by fixing the parameters to such values is then an approximation to the ground state wavefunction, and the expectation value of the energy in that state is an upper bound to the ground state energy. **The [Hartree–Fock method](https://en.m.wikipedia.org/wiki/Hartree%E2%80%93Fock_method), [Density matrix renormalization group](https://en.m.wikipedia.org/wiki/Density_matrix_renormalization_group), and [Ritz method](https://en.m.wikipedia.org/wiki/Ritz_method) apply the variational method**.

  * This principle relies on the fact that a Hermitian is equal to its own conjugate transpose (ie 𝐻=𝐻†), and since any eigenvalue of H must be real, all eigenvalues of 𝐻 have the property of 𝜆𝑖=𝜆∗.

###### **Mean-field theory**

* In physics and probability theory, [mean-field theory](https://en.m.wikipedia.org/wiki/Mean-field_theory) (aka MFT or rarely self-consistent field theory) studies the behavior of high-dimensional random (stochastic) models by studying a simpler model that approximates the original by averaging over degrees of freedom (the number of values in the final calculation of a statistic that are free to vary). Such models consider many individual components that interact with each other. 

* In MFT, the effect of all the other individuals on any given individual is approximated by a single averaged effect, **thus reducing a [many-body problem](https://en.m.wikipedia.org/wiki/Many-body_problem) to a [one-body problem (two body)](https://en.m.wikipedia.org/wiki/Two-body_problem)**.

###### **Hartree–Fock Method**

* In computational physics and chemistry, the [Hartree–Fock (HF) method](https://en.m.wikipedia.org/wiki/Hartree%E2%80%93Fock_method) is a method of approximation for the determination of the wave function and the energy of a [quantum many-body system](https://en.m.wikipedia.org/wiki/Many-body_problem) <u>**in a stationary state**</u>.

* The Hartree–Fock method often assumes that the exact N-body wave function of the system can be approximated by a single [Slater determinant](https://en.m.wikipedia.org/wiki/Slater_determinant) (in the case where the particles are fermions) or by a single [permanent](https://en.m.wikipedia.org/wiki/Permanent_(mathematics)) (in the case of bosons) of N [spin-orbitals](https://en.m.wikipedia.org/wiki/Atomic_orbital). 

* By invoking the [variational method](https://en.m.wikipedia.org/wiki/Calculus_of_variations), one can derive a set of N-coupled equations for the N spin orbitals. A solution of these equations yields the Hartree–Fock wave function and energy of the system.





https://en.m.wikipedia.org/wiki/Hartree%E2%80%93Fock_method

https://en.m.wikipedia.org/wiki/Slater_determinant

https://en.m.wikipedia.org/wiki/Permanent_(mathematics)

https://quantumai.google/cirq/experiments/hfvqe

###### **Density matrix renormalization group**

https://en.m.wikipedia.org/wiki/Density_matrix_renormalization_group

###### **Ritz Method**

https://www.youtube.com/watch?v=w4ijyrdLbNA

https://en.m.wikipedia.org/wiki/Ritz_method

https://en.m.wikipedia.org/wiki/Rayleigh%E2%80%93Ritz_method

#### <font color="blue">**Variational Quantum Algorithms**

https://arxiv.org/abs/2012.09265

###### <font color="blue">**Variational Quantum Algorithms**

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/quantum_187.png)

Source: [Variational Quantum Algorithms](https://www.youtube.com/watch?v=YtepXvx5zdI)

###### <font color="blue">**Ansatz**

> In the context of variational circuits, an ansatz usually describes a subroutine consisting of a sequence of gates applied to specific wires. Similar to the architecture of a neural network, this only defines the base structure, while the types of gates and/or their free parameters can be optimized by the variational procedure. 

One can distinguish three different base structures, namely 

* a layered gate ansatz, 

* an alternating operator ansatz, and 

* a tensor network ansatz.

The ansatz can come from:
- Some basis in physics, chemistry, or quantum information theory (e.g., VQE)
- The structure of the problem (e.g., QAOA)
- Intuition borrowed from machine learning
- No place at all (use your imagination!)

The choice of ansatz affects the model/function that can be learned (more layers often better)

https://pennylane.ai/qml/glossary/circuit_ansatz.html

https://medium.com/arnaldo-gunzi-quantum/what-is-ansatz-31e682b0518b

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/quantum_188.png)

https://en.m.wikipedia.org/wiki/Ansatz

https://de.m.wikipedia.org/wiki/Ansatz_(Mathematik)

###### <font color="blue">**Embedd Classical Data into a Quantum Variational Circuit**

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/quantum_189.png)

> **The easiest way for a parameter to enter a circuit is through a rotation of a single qubit, in proportion to the value of a single datapoint, so a single scalar value:**

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/quantum_190.png)

> **You can also use a sequence of rotations to embedd data (reuploding).** And maybe there is free parameters in between as well. Can make a more complex function available than if you upload only once in a single rotation.

> **Learnable embeddings**: The other idea is to actually have a trainable embedding layer. Not to worry about training the unitary of the circuit, but worry about training the embedding and then use standard quantum information metrics to classify the data.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/quantum_191.png)

###### <font color="blue">**Optimizer**

> When we're using variational circuits, if you can compute the gradients the parameter shift rule, then that opens up every possible flavor of gradient descent. Like standard gradient descent, but in deep learning also all sorts of gradient descent optimizers. Most common: momentum and Adam.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/quantum_192.png)

<font color="blue">**But there are also quantum-aware optimizers!**

**Rotosolve**: don't use gradient descent, but sinusoid structure. You eventually end up in a global minima via a sequence of individual jumps to local minima.

[**Quantum Natural Optimizer**](https://pennylane.ai/qml/demos/tutorial_quantum_natural_gradient.html): Internet geometry of quantum circuits and quantum physical system is not Euclidean like the world around us, like this rectilinear structure. It's more sinusoidal structure. We should take this into account. 

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/quantum_193.png)

<font color="blue">**Barren Plateau**

* There are parts of the optimization landscape where the gradient is zero, and around it also zero, like in classical deep learning. everything flat in every direction where you go.

* Barren Plateau can come from number of different effects: choice from circuit ansatz, choice of parametrization (parameter values), or from cost function

* several proposals to overcome this

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/quantum_194.png)

#### <font color="blue">**VQE: Mathematics**

> Model different molecules and how they interact together

* **Objective**: create new, more effective medicines along with researching how medications might react with diseases such as cancer.

* **Challenge**: effectively make the calculations needed for the chemical reactions that occur. 

  * One such calculation is **calculating the ground state energy of different small molecules**.

  * Running this calculation on a classical computer can take exponential time to get an exact numerical answer.


* *Exkurs: The [ground state energy](https://en.wikipedia.org/wiki/Excited_state) is the lowest possible energy state that a molecule can be in. This is important because it gives more information about the electron configuration of the molecule.*

![gg](https://upload.wikimedia.org/wikipedia/commons/thumb/a/a8/Energy_levels.svg/306px-Energy_levels.svg.png)


**From chemistry and physics we know that the Schrodinger equation tells us that if $H$ is the Hamiltonian of a system, $H|ψ⟩ = E_g|ψ⟩$ where the minimum eigenvalue $E_g$ that solves this equation corresponds to the ground state energy of the molecule**.

* To find this energy, we want to **find the exact minimum eigenvalue** that corresponds to that Hermitian matrix. 

* However that **would require a perfectly working quantum computer using the QPE (Quantum Phase Estimation Algorithm)**, which is not possible as of right now.

* Rather, we can use the [variational principle](https://en.wikipedia.org/wiki/Variational_principle) to find a close, if not exact solution.






This principle relies on the fact that a Hermitian is equal to its own conjugate transpose (ie 𝐻=𝐻†), 

* and since any eigenvalue of H must be real, all eigenvalues of 𝐻 have the property of 𝜆𝑖=𝜆∗. 

* We can rewrite the Hermitian as follows, where 𝜆 is the eigenvalue associated with the eigenvector |𝜓_i⟩

> $H=\sum_{i=1}^{N} \lambda_{i}\left|\psi_{i}\right\rangle\left\langle\psi_{i}\right|$

* which we can then use to define the expectation of this Hermitian as

> $\langle H\rangle_{\psi} \equiv\langle\psi|H| \psi\rangle$

* And finally we can substitute in for $H$ and get the following

> $\begin{aligned}\langle H\rangle_{\psi}=\langle\psi|H| \psi\rangle &=\left\langle\psi\left|\left(\sum_{i=1}^{N} \lambda_{i}\left|\psi_{i}\right\rangle\left\langle\psi_{i}\right|\right)\right| \psi\right\rangle \\ &=\sum_{i=1}^{N} \lambda_{i}\left\langle\psi \mid \psi_{i}\right\rangle\left\langle\psi_{i} \mid \psi\right\rangle \\ &=\sum_{i=1}^{N} \lambda_{i}\left|\left\langle\psi_{i} \mid \psi\right\rangle\right|^{2} \end{aligned}$

* which demonstrates that **any state can be described using a sum of the eigenvalues of the Hermitian as weights**. 

* One final property of the Hermitian is what truly allows the variational principle to work is that the eigenvalues of it have to be real, non-negative numbers, such that

> $\lambda_{\min } \leq\langle H\rangle_{\psi}=\langle\psi|H| \psi\rangle$

* And this equation is what gives power to the Variational Quantum Eigensolver. 

> **Since the minimum eigenvalue can never be smaller than the expectation value that we are looking for, then by minimizing our expectation, we will get really close to the minimum eigenvalue**.

https://medium.com/mit-6-s089-intro-to-quantum-computing/quantum-molecule-simulation-using-vqe-d34b9c651e3d

#### <font color="blue">**VQE: Algorithm**

This algorithm is a **hybrid machine learning quantum computer algorithm**, where we run part of it on a quantum computer, and then finish off the minimization classically (since a solely quantum algorithm provides no speedups).

**Step 1: We encode the molecular Hamiltonian onto the qubits**

* the variational form typically uses the U3 gate (QASM?), where we can describe it with the following matrix:

> $U 3(\theta, \phi, \lambda)=\left(\begin{array}{cc}\cos \left(\frac{\theta}{2}\right) & -e^{i \lambda} \sin \left(\frac{\theta}{2}\right) \\ e^{i \phi} \sin \left(\frac{\theta}{2}\right) & e^{i \lambda+i \phi} \cos \left(\frac{\theta}{2}\right)\end{array}\right)$

* We can then construct a quantum circuit that encodes the information that we want, but it is important that this circuit (and these gates as a whole) allow for any universal state to be entered and returned, which can result in more complex circuits.

* The end result of this encoding allows us to make calculations with the qubits, but now the qubits are mimicking the electron orbital interactions and so it almost as if we are making the computations we need with the molecule itself.



![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/quantum_181.png)

**Step 2: Create the ansatz**

* An ansatz, in simplest terms, is a really good mathematical guess that can cover many possible states. 

* For our purposes, the thing that we will be guessing will be what the electron configuration is for the specific molecule at a given interatomic distance between each atom and thus the ansatz can be described as our trial wavefunction. 

* We also require that the ansatz be shallow due to the nature of programs on quantum computers needing to run quickly or else they’ll be more prone to errors.

> In the context of variational circuits, an ansatz usually describes a subroutine consisting of a sequence of gates applied to specific wires. Similar to the architecture of a neural network, this only defines the base structure, while the types of gates and/or their free parameters can be optimized by the variational procedure. **One can distinguish three different base structures, namely a layered gate ansatz, an alternating operator ansatz, and a tensor network ansatz.**

https://pennylane.ai/qml/glossary/circuit_ansatz.html

https://medium.com/arnaldo-gunzi-quantum/what-is-ansatz-31e682b0518b


![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/quantum_183.png)

**Step 3: Calculate our trial state energy**

* use the information that we have to calculate the ground state energy level of our ansatz. 

* The calculation uses the information given by the Hamiltonian for a given inter-atomic distance, and the energy for that particular electron configuration.

**Step 4: The Classical Side**

> Task a: Measure and optimize our parameter

* The first step here is to measure our qubits such that we get the information that we wanted from the quantum steps. 

* We then use machine learning algorithms to optimize our parameters such that our cost function, which represents the ground state energy, is minimized.

* In terms of the optimization, the [Simultaneous Perturbation Stochastic Approximation optimizer (SPSA)](https://en.m.wikipedia.org/wiki/Simultaneous_perturbation_stochastic_approximation) is typically used due to how noisy the objective function can be. One thing that we had considered using during our experimentation with the VQE algorithm was gradient descent. 

* We noticed that our original code was just an iteration over the possible values that the inter-atomic distance could take and we thought that gradient descent might be able to speed this process up by just choosing a good start value. 

* However, what we found was that this strategy was prone to getting stuck in local optimum and through the reading of the Qiskit textbook and other research papers, we found that it is in fact a costly optimization in terms of the number of circuit evaluations performed.

* So then we questioned how does SPSA work and why does it not have the same pitfalls as gradient descent? They both share the similarity that both optimizes approximate the gradient of our cost function, however, SPSA “does so by concurrently perturbing all of the parameters in a random fashion, in contrast to gradient descent where each parameter is perturbed independently” as discussed in the Qiskit textbook tutorial.



> Task b: Iterate using the resulting parameters

* The second step of the classical side of this algorithm is that we then take the optimized parameters that we got and feed them into the quantum part again, giving us a new set of parameters that should improve the ansatz. 

* This step is iterative because we continually run through out optimizer until we are able to converge onto the lowest energy state for that bond length. 

* We then continue to iterate over the different inter-atomic distances until we find the bond length associated with the molecule’s lowest energy state.


*Results of VQE on RbH where Hartree-Fock is the expected and VQE is what we got from running the algorithm:*

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/quantum_184.png)

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/quantum_185.png)

Source: [Variational Quantum Algorithms](https://www.youtube.com/watch?v=YtepXvx5zdI)

#### <font color="blue">**QAOA (Quantum Approximate Optimization Algorithm)**

https://qiskit.org/textbook/ch-applications/qaoa.html

https://en.wikipedia.org/wiki/Quantum_optimization_algorithms

* the cost function is something that is encoding an optimization problem. you might have an (discrete) optimization problem with many clauses that have to be satisfied and you have to encode this into an Ising type model, a spin chain type model, which then can be converted into observables which can be measured on a quantum circuit

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/quantum_186.png)

Source: [Variational Quantum Algorithms](https://www.youtube.com/watch?v=YtepXvx5zdI)